In [27]:
import sys
import os
sys.path.insert(1, '..')

import numpy as np
import scipy as sp
import pandas as pd
from utils import *

import librosa

import matplotlib.pyplot as plt
import seaborn as sns

import IPython.display as ipd

In [2]:
speech, _ = librosa.load('../dados/falas_04/resampled_rate/F180901.WAV', sr = None) # Downsample 44.1kHz to 8kHz

In [3]:
noise, _ = librosa.load('../dados/ruidos_04/resampled_rate/MIX MASTER-start-run-stop.flac', sr = None) # Downsample 44.1kHz to 8kHz

In [4]:
speech_len = len(speech)
noise_len = len(noise)

# Noise cut will be speech length + 16000 points (2s)
cut_len = speech_len + 16000
print(cut_len)

# Pega o ultimo valor possivel para fazer o recorte da amostragem
last_value = len(noise[:-cut_len])

cut_start = np.random.randint(0, last_value)
cut_end = cut_start + cut_len

noise_cut = noise[cut_start:cut_end]

43629


### Original Noise

In [5]:
ipd.Audio(noise, rate= 8000)

### Noise cut

In [6]:
ipd.Audio(noise_cut, rate= 8000)

### Speech

In [7]:
ipd.Audio(speech, rate= 8000)

In [42]:
def cut_noise(speech: list, noise: list, pure_noise_len = 16000) -> list:
    """ """
    
    speech_len = len(speech)
    noise_len = len(noise)

    # Noise cut will be speech length + 16000 points (2s)
    cut_len = speech_len + pure_noise_len

    # Pega o ultimo valor possivel para fazer o recorte da amostragem
    last_value = len(noise[:-cut_len])

    cut_start = np.random.randint(0, last_value)
    cut_end = cut_start + cut_len
    noise_cut = noise[cut_start:cut_end]
    
    return noise_cut, cut_start

In [43]:
noise_cut2, real = cut_noise(speech, noise)

In [44]:
real

31556

### Pad Beggining of Speech

In [10]:
x_len = len(speech)
y_len = len(noise_cut2)

len_delta = y_len - x_len 

In [11]:
len_delta

16000

In [12]:
noise_cut

array([ 0.05822754,  0.07119751, -0.01455688, ...,  0.13641357,
        0.05709839, -0.05822754], dtype=float32)

In [13]:
zero_padding = [0] * len_delta

In [14]:
concatenated = np.concatenate((zero_padding, speech), axis = None)

In [15]:
contaminated = noise_cut + concatenated

In [16]:
def zero_pad_beggining(bigger_signal: list, smaller_signal: list) -> list:
    """ """
    
    bigger_len = len(bigger_signal)
    smaller_len = len(smaller_signal)
    len_delta = bigger_len - smaller_len
    
    zero_padding = [0] * len_delta
    padded = np.concatenate((zero_padding, smaller_signal), axis = None)
    
    return padded

In [17]:
padded_speech = zero_pad_beggining(noise_cut, speech)

In [18]:
contaminated = padded_speech + noise_cut

In [20]:
ipd.Audio(contaminated, rate = 8000)

In [22]:
ipd.Audio(contaminated[16000:], rate = 8000)

In [29]:
actual_snr = snr_db(speech, noise_cut[16000:])
print('Actual SNR:', actual_snr)
desired_snr = 10

Actual SNR: -9.694851141523966


In [30]:
out_noise = fix_snr(noise_cut, actual_snr, desired_snr)

In [31]:
actual_snr = snr_db(speech, out_noise)

In [33]:
contaminated_desired = padded_speech + out_noise

In [34]:
ipd.Audio(contaminated_desired, rate = 8000)